# 41 データの読み込み

In [1]:
import pandas as pd
customer = pd.read_csv("customer_join.csv")
uselog_months = pd.read_csv("use_log_months.csv")

## 当月・前月の利用回数を集計

In [2]:
year_maonth = list(uselog_months["年月"].unique())
uselog = pd.DataFrame()
for i in range(1, len(year_maonth)):
    #当月の利用回数
    tmp = uselog_months.loc[uselog_months["年月"]==year_maonth[i]]
    tmp.rename(columns={"count":"count_0"}, inplace=True)
    #前月の利用回数
    tmp_before = uselog_months.loc[uselog_months["年月"]==year_maonth[i-1]]
    del tmp_before["年月"]
    tmp_before.rename(columns={"count":"count_1"}, inplace=True)
    tmp = pd.merge(tmp, tmp_before, on="customer_id", how="left")
    uselog = pd.concat([uselog, tmp], ignore_index=True)
uselog.head()

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,年月,customer_id,count_0,count_1
0,201805,AS002855,5,4.0
1,201805,AS009373,4,3.0
2,201805,AS015233,7,NaN
3,201805,AS015315,3,6.0
4,201805,AS015739,5,7.0


# 42 退会前月の退会顧客データを作成

In [3]:
from dateutil.relativedelta import relativedelta
exit_customer = customer.loc[customer["is_deleted"]==1]
exit_customer["exit_date"] = None
exit_customer["end_date"] = pd.to_datetime(exit_customer["end_date"])
for i in range(len(exit_customer)):
    # 退会月の-1月を"exit_date"とする
    exit_customer["exit_date"].iloc[i] = exit_customer["end_date"].iloc[i] - relativedelta(months=1)
exit_customer["年月"] = exit_customer["exit_date"].dt.strftime("%Y%m")
uselog["年月"] = uselog["年月"].astype(str)
exit_uselog = pd.merge(uselog, exit_customer, on=["customer_id", "年月"], how="left")
print(len(uselog))
exit_uselog.head()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveat

33851


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
0,201805,AS002855,5,4.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,201805,AS009373,4,3.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,201805,AS015233,7,NaN,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,201805,AS015315,3,6.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,201805,AS015739,5,7.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


退会したcustomerのレコードのみ抽出(欠損値を除外)

In [4]:
exit_uselog = exit_uselog.dropna(subset=["name"])
print(len(exit_uselog))
print(len(exit_uselog["customer_id"].unique()))
exit_uselog.head()

1104
1104


,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
19,201805,AS055680,3,3.0,XXXXX,C01,M,2018-03-01,2018-06-30,CA1,...,10500.0,通常,3.000000,3.0,3.0,3.0,0.0,2018-06-30,3.0,2018-05-30 00:00:00
57,201805,AS169823,2,3.0,XX,C01,M,2017-11-01,2018-06-30,CA1,...,10500.0,通常,3.000000,3.0,4.0,2.0,1.0,2018-06-30,7.0,2018-05-30 00:00:00
110,201805,AS305860,5,3.0,XXXX,C01,M,2017-06-01,2018-06-30,CA1,...,10500.0,通常,3.333333,3.0,5.0,2.0,0.0,2018-06-30,12.0,2018-05-30 00:00:00
128,201805,AS363699,5,3.0,XXXXX,C01,M,2018-02-01,2018-06-30,CA1,...,10500.0,通常,3.333333,3.0,5.0,2.0,0.0,2018-06-30,4.0,2018-05-30 00:00:00
147,201805,AS417696,1,4.0,XX,C03,F,2017-09-01,2018-06-30,CA1,...,6000.0,通常,2.000000,1.0,4.0,1.0,0.0,2018-06-30,9.0,2018-05-30 00:00:00


# 43 継続顧客のデータを作成

In [5]:
conti_customer = customer.loc[customer["is_deleted"]==0]
conti_uselog = pd.merge(uselog, conti_customer, on=["customer_id"], how="left")
print(len(conti_uselog))
conti_uselog = conti_uselog.dropna(subset=["name"])
print(len(conti_uselog))

33851
27422


カスタマ一人あたり、各月の利用回数レコードとなるので、1人あたりランダム月の利用回数を取り出す

In [6]:
# データのシャッフル すべての行数(flac=100%)をランダムにサンプリングしていく
conti_uselog = conti_uselog.sample(frac=1).reset_index(drop=True)
conti_uselog = conti_uselog.drop_duplicates(subset="customer_id")
print(len(conti_uselog))
conti_uselog.head()

2842


,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period
0,201807,PL227814,9,10.0,XXXXX,C01,M,2018-04-05,NaN,CA3,...,オールタイム,10500.0,入会費無料,7.250000,7.0,10.0,3.0,1.0,2019-04-30,12.0
1,201902,TS621030,4,2.0,XXXX,C02,F,2015-09-01,NaN,CA1,...,デイタイム,7500.0,通常,3.916667,4.0,5.0,2.0,1.0,2019-04-30,43.0
2,201903,HD849426,5,7.0,XX,C03,M,2017-08-01,NaN,CA2,...,ナイト,6000.0,入会費半額,6.416667,6.5,9.0,5.0,1.0,2019-04-30,20.0
3,201901,HD872512,5,8.0,XXXXX,C03,M,2018-07-03,NaN,CA1,...,ナイト,6000.0,通常,6.888889,7.0,9.0,5.0,1.0,2019-04-30,9.0
4,201807,PL147603,3,4.0,XXXXX,C03,M,2016-05-01,NaN,CA1,...,ナイト,6000.0,通常,4.000000,4.0,8.0,2.0,1.0,2019-04-30,35.0


In [7]:
predict_data = pd.concat([conti_uselog, exit_uselog], ignore_index=True)
print(len(predict_data))
predict_data.head()

3946


,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
0,201807,PL227814,9,10.0,XXXXX,C01,M,2018-04-05,NaN,CA3,...,10500.0,入会費無料,7.250000,7.0,10.0,3.0,1.0,2019-04-30,12.0,NaN
1,201902,TS621030,4,2.0,XXXX,C02,F,2015-09-01,NaN,CA1,...,7500.0,通常,3.916667,4.0,5.0,2.0,1.0,2019-04-30,43.0,NaN
2,201903,HD849426,5,7.0,XX,C03,M,2017-08-01,NaN,CA2,...,6000.0,入会費半額,6.416667,6.5,9.0,5.0,1.0,2019-04-30,20.0,NaN
3,201901,HD872512,5,8.0,XXXXX,C03,M,2018-07-03,NaN,CA1,...,6000.0,通常,6.888889,7.0,9.0,5.0,1.0,2019-04-30,9.0,NaN
4,201807,PL147603,3,4.0,XXXXX,C03,M,2016-05-01,NaN,CA1,...,6000.0,通常,4.000000,4.0,8.0,2.0,1.0,2019-04-30,35.0,NaN


# 44 予測する月の在籍期間を作成

In [8]:
predict_data["period"] = 0
predict_data["now_date"] = pd.to_datetime(predict_data["年月"], format="%Y%m")
predict_data["start_date"] = pd.to_datetime(predict_data["start_date"])
for i in range(len(predict_data)):
    delta = relativedelta(predict_data["now_date"][i], predict_data["start_date"][i])
    predict_data["period"][i] = int(delta.years*12 + delta.months)
predict_data.head()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date,period,now_date
0,201807,PL227814,9,10.0,XXXXX,C01,M,2018-04-05,NaN,CA3,...,7.250000,7.0,10.0,3.0,1.0,2019-04-30,12.0,NaN,2,2018-07-01
1,201902,TS621030,4,2.0,XXXX,C02,F,2015-09-01,NaN,CA1,...,3.916667,4.0,5.0,2.0,1.0,2019-04-30,43.0,NaN,41,2019-02-01
2,201903,HD849426,5,7.0,XX,C03,M,2017-08-01,NaN,CA2,...,6.416667,6.5,9.0,5.0,1.0,2019-04-30,20.0,NaN,19,2019-03-01
3,201901,HD872512,5,8.0,XXXXX,C03,M,2018-07-03,NaN,CA1,...,6.888889,7.0,9.0,5.0,1.0,2019-04-30,9.0,NaN,5,2019-01-01
4,201807,PL147603,3,4.0,XXXXX,C03,M,2016-05-01,NaN,CA1,...,4.000000,4.0,8.0,2.0,1.0,2019-04-30,35.0,NaN,26,2018-07-01


# 欠損値の除去

In [9]:
predict_data.isna().sum()

年月                      0
customer_id             0
count_0                 0
count_1               250
name                    0
class                   0
gender                  0
start_date              0
end_date             2842
campaign_id             0
is_deleted              0
class_name              0
price                   0
campaign_name           0
mean                    0
median                  0
max                     0
min                     0
routine_flg             0
calc_date               0
membership_period       0
exit_date            2842
period                  0
now_date                0
dtype: int64

In [10]:
predict_data = predict_data.dropna(subset=["count_1"])
predict_data.isna().sum()

年月                      0
customer_id             0
count_0                 0
count_1                 0
name                    0
class                   0
gender                  0
start_date              0
end_date             2644
campaign_id             0
is_deleted              0
class_name              0
price                   0
campaign_name           0
mean                    0
median                  0
max                     0
min                     0
routine_flg             0
calc_date               0
membership_period       0
exit_date            2644
period                  0
now_date                0
dtype: int64